In [ ]:
import os
import math
import json
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
from concurrent.futures import ProcessPoolExecutor, as_completed

# ==========
# 线程超卖控制（重要）
# ==========
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")

# 如果你的文件系统锁会出问题，可尝试：
os.environ.setdefault("HDF5_USE_PATH_LOCKING", "FALSE")


'FALSE'

In [6]:
# ======================
# PATH CONFIG
# ======================
NOTEBOOK_DIR = Path().resolve()
ROOT_DIR = (NOTEBOOK_DIR / "../data").resolve()
RUN_DIR = ROOT_DIR / "run"
CWD_PATH  = RUN_DIR / "CWD.nc"
dEF_PATH  = ROOT_DIR / "ET/dEF.nc"
dSIF_PATH = ROOT_DIR / "SIF/dSIF.nc"

OUT_GLOBAL_DIR = ROOT_DIR / "veg_activity"
OUT_GLOBAL_DIR.mkdir(parents=True, exist_ok=True)

OUT_TILE_DIR = RUN_DIR / "veg_activity/tiles"
OUT_TILE_DIR.mkdir(parents=True, exist_ok=True)

# ======================
# VARIABLE NAMES
# ======================
VAR_CWD  = "CWD"
VAR_dEF  = "dEF"
VAR_dSIF = "dSIF"

# ======================
# TILE CONFIG
# ======================
TILE_Y = 300   # lat block
TILE_X = 300   # lon block

# ======================
# METHOD PARAMS
# ======================
NBINS = 50
Q = 0.90

MIN_EVENT_STEPS = 1       # >=8天; 你的数据是8-day，所以1 step = 8天
MINVALIDBINS = 8
MINPOINTSTOTAL = 30

RUNAWAY_YEARS = 5         # 连续>=5年无CWD<=0重置 -> discard pixel

# 缺测值处理（你的 dEF/dSIF 是 -9999）
FILL_X = -9999.0

# 输出压缩
ENC_2D = dict(zlib=True, complevel=4, dtype="float32")


In [3]:
ds_cwd  = xr.open_dataset(CWD_PATH,  engine="netcdf4")
ds_def  = xr.open_dataset(dEF_PATH,  engine="netcdf4")
ds_dsif = xr.open_dataset(dSIF_PATH, engine="netcdf4")

print(ds_cwd)
print(ds_def)
print(ds_dsif)

# 取 time/lat/lon，确保一致
time = ds_cwd["time"].values
lat  = ds_cwd["lat"].values
lon  = ds_cwd["lon"].values

assert ds_def["time"].size == ds_cwd["time"].size
assert ds_dsif["time"].size == ds_cwd["time"].size
assert ds_def["lat"].size  == ds_cwd["lat"].size
assert ds_def["lon"].size  == ds_cwd["lon"].size

print("time:", time[0], "->", time[-1], "len=", len(time))
print("lat:", lat[0], lat[-1], "n=", len(lat))
print("lon:", lon[0], lon[-1], "n=", len(lon))


<xarray.Dataset>
Dimensions:  (time: 828, lat: 3600, lon: 7200)
Coordinates:
  * time     (time) datetime64[ns] 2003-01-01 2003-01-09 ... 2020-12-26
  * lat      (lat) float32 -89.97 -89.93 -89.88 -89.82 ... 89.88 89.93 89.97
  * lon      (lon) float32 -180.0 -179.9 -179.9 -179.8 ... 179.9 179.9 180.0
Data variables:
    CWD      (time, lat, lon) float32 ...
<xarray.Dataset>
Dimensions:  (time: 828, lon: 7200, lat: 3600)
Coordinates:
  * time     (time) datetime64[ns] 2003-01-01 2003-01-09 ... 2020-12-26
  * lon      (lon) float64 -180.0 -179.9 -179.9 -179.8 ... 179.9 179.9 180.0
  * lat      (lat) float64 -89.97 -89.92 -89.88 -89.83 ... 89.88 89.93 89.98
Data variables:
    crs      |S1 ...
    dEF      (time, lat, lon) float32 ...
Attributes:
    CDI:                 Climate Data Interface version 2.5.0 (https://mpimet...
    Conventions:         CF-1.5
    GDAL_AREA_OR_POINT:  Area
    GDAL:                GDAL 3.10.3, released 2025/04/01
    history:             Fri Jan 09 21:33:58

In [4]:
def build_year_slices(time_index: np.ndarray) -> np.ndarray:
    """
    return int32 array shape (n_years, 2) with [start, end) indices
    """
    t = pd.to_datetime(time_index)
    years = t.year.values
    uniq_years = np.unique(years)
    slices = []
    for y in uniq_years:
        idx = np.where(years == y)[0]
        if idx.size == 0:
            continue
        slices.append((int(idx[0]), int(idx[-1] + 1)))
    return np.array(slices, dtype=np.int32)

YEAR_SLICES = build_year_slices(time)
print("N years:", YEAR_SLICES.shape[0], "first/last:", YEAR_SLICES[0], YEAR_SLICES[-1])


N years: 18 first/last: [ 0 46] [782 828]


In [7]:
NY = len(lat)
NX = len(lon)

def make_tiles(ny, nx, tile_y, tile_x):
    tiles = []
    for y0 in range(0, ny, tile_y):
        y1 = min(ny, y0 + tile_y)
        for x0 in range(0, nx, tile_x):
            x1 = min(nx, x0 + tile_x)
            tiles.append((y0, y1, x0, x1))
    return tiles

TILES = make_tiles(NY, NX, TILE_Y, TILE_X)
print("Total tiles:", len(TILES))
print("Example tile:", TILES[0])


Total tiles: 288
Example tile: (0, 300, 0, 300)


In [8]:
def load_tile(y0, y1, x0, x1):
    cwd  = ds_cwd[VAR_CWD].isel(lat=slice(y0,y1), lon=slice(x0,x1)).transpose("time","lat","lon").load()
    dEF  = ds_def[VAR_dEF].isel(lat=slice(y0,y1), lon=slice(x0,x1)).transpose("time","lat","lon").load()
    dSIF = ds_dsif[VAR_dSIF].isel(lat=slice(y0,y1), lon=slice(x0,x1)).transpose("time","lat","lon").load()

    # 缺测：-9999 -> NaN
    dEF  = dEF.where(dEF != FILL_X)
    dSIF = dSIF.where(dSIF != FILL_X)

    # 转 float32（减内存/加速）
    return (cwd.data.astype(np.float32),
            dEF.data.astype(np.float32),
            dSIF.data.astype(np.float32))

# 试一个 tile
y0,y1,x0,x1 = TILES[0]
cwd_t, def_t, dsif_t = load_tile(y0,y1,x0,x1)
print(cwd_t.shape, def_t.shape, dsif_t.shape)
print("CWD finite ratio:", np.isfinite(cwd_t).mean())
print("dEF finite ratio:", np.isfinite(def_t).mean())
print("dSIF finite ratio:", np.isfinite(dsif_t).mean())


(828, 300, 300) (828, 300, 300) (828, 300, 300)
CWD finite ratio: 1.0
dEF finite ratio: 0.0
dSIF finite ratio: 3.6500268384326356e-06


In [ ]:
# cal_S0.py
# Core S0 estimation (Stocker-style binning) with lightweight QC outputs
# - Adds: (1) slope significance (one-sided) (2) piecewise 1-change-point BIC test
# - Returns: S0, p_one, bic_lin, bic_seg, cp, flag
#
# flag:
#   0 = invalid / filtered
#   1 = linear accepted => S0 output
#   2 = segmented model clearly better (change point / flattening) => S0 not output
#
# Notes:
# - p-value uses Normal approximation for speed & numba-compatibility
# - piecewise model uses brute-force change point search on binned points

import math
import numpy as np
import numba as nb


@nb.njit(cache=True)
def _event_window_one_year(cwd_year, min_event_steps):
    """
    Identify the single largest CWD event window in one year.
    cwd_year: 1D float32, length Ty (CWD >= 0 in deficit period; reset near 0 in wet)
    return (ok, start, end_exclusive, cwdmax)

    Stocker-style trimming:
    - peak at maximum CWD
    - start at last CWD<=0 before peak, but we will shift +1 to avoid including reset point
    - end at first time after peak when CWD < 0.9*CWDmax (rewetting removal)
    """
    Ty = cwd_year.size

    # peak (max CWD)
    peak = -1
    cwdmax = -1e30
    for i in range(Ty):
        v = cwd_year[i]
        if np.isfinite(v) and v > cwdmax:
            cwdmax = v
            peak = i
    if peak < 0 or (not np.isfinite(cwdmax)) or cwdmax <= 0.0:
        return 0, 0, 0, 0.0

    # start: last CWD<=0 before peak  (then +1 to exclude reset point)
    start = 0
    reset_idx = -1
    for i in range(peak, -1, -1):
        v = cwd_year[i]
        if np.isfinite(v) and v <= 0.0:
            reset_idx = i
            break
    if reset_idx >= 0:
        start = reset_idx + 1
        if start >= Ty:
            return 0, 0, 0, 0.0
    else:
        # no reset found; start from 0
        start = 0

    # end: first time after peak when CWD < 0.9*CWDmax
    thr = 0.9 * cwdmax
    end = Ty
    for i in range(peak, Ty):
        v = cwd_year[i]
        if np.isfinite(v) and v < thr:
            end = i
            break

    if end - start < min_event_steps:
        return 0, 0, 0, 0.0

    return 1, start, end, cwdmax


@nb.njit(cache=True)
def _count_runaway_years(cwd, year_slices):
    """
    Count max consecutive years with no reset (no CWD<=0 in that year).
    Stocker: remove pixels where accumulation extends > 5 years.
    """
    nY = year_slices.shape[0]
    max_consec = 0
    consec = 0
    for yi in range(nY):
        s = year_slices[yi, 0]
        e = year_slices[yi, 1]
        has_reset = False
        for t in range(s, e):
            v = cwd[t]
            if np.isfinite(v) and v <= 0.0:
                has_reset = True
                break
        if has_reset:
            consec = 0
        else:
            consec += 1
            if consec > max_consec:
                max_consec = consec
    return max_consec


# -------------------------
# Normal CDF (fast p-value approx)
# Phi(z) = 0.5 * erfc(-z / sqrt(2))
# -------------------------
@nb.njit(cache=True)
def _norm_cdf(z):
    return 0.5 * math.erfc(-z / 1.4142135623730951)


# -------------------------
# OLS fit: y = a + b*x
# return (ok, a, b, rss, se_b, t_b, p_one_sided)
# p_one_sided: H1: b < 0  (Normal approx)
# -------------------------
@nb.njit(cache=True)
def _ols_fit(x, y):
    n = x.size
    if n < 3:
        return 0, 0.0, 0.0, 1e30, 0.0, 0.0, 1.0

    sx = 0.0
    sy = 0.0
    sxx = 0.0
    sxy = 0.0
    for i in range(n):
        sx += x[i]
        sy += y[i]
        sxx += x[i] * x[i]
        sxy += x[i] * y[i]

    denom = n * sxx - sx * sx
    if denom == 0.0 or (not np.isfinite(denom)):
        return 0, 0.0, 0.0, 1e30, 0.0, 0.0, 1.0

    b = (n * sxy - sx * sy) / denom
    a = (sy - b * sx) / n

    if (not np.isfinite(a)) or (not np.isfinite(b)):
        return 0, 0.0, 0.0, 1e30, 0.0, 0.0, 1.0

    # RSS
    rss = 0.0
    for i in range(n):
        r = y[i] - (a + b * x[i])
        rss += r * r
    if (not np.isfinite(rss)) or rss <= 0.0:
        rss = 1e-30

    # Standard error of b
    xbar = sx / n
    sxxc = 0.0
    for i in range(n):
        dx = x[i] - xbar
        sxxc += dx * dx

    if sxxc <= 0.0 or (not np.isfinite(sxxc)):
        return 0, a, b, rss, 0.0, 0.0, 1.0

    s2 = rss / (n - 2)  # residual variance
    if (not np.isfinite(s2)) or s2 <= 0.0:
        return 0, a, b, rss, 0.0, 0.0, 1.0

    se_b = math.sqrt(s2 / sxxc)
    if (not np.isfinite(se_b)) or se_b <= 0.0:
        return 0, a, b, rss, 0.0, 0.0, 1.0

    t_b = b / se_b

    # One-sided p for H1: b < 0
    # Normal approx
    p_one = _norm_cdf(t_b)
    if not np.isfinite(p_one):
        p_one = 1.0

    return 1, a, b, rss, se_b, t_b, p_one


@nb.njit(cache=True)
def _bic_from_rss(rss, n, k):
    # BIC = n * ln(RSS/n) + k * ln(n)
    if rss <= 0.0:
        rss = 1e-30
    return n * math.log(rss / n) + k * math.log(n)


# -------------------------
# Best 1-change-point piecewise linear:
# split at cp index (left: [0:cp], right: [cp:n])
# return:
# (ok, best_cp, rss_total, bic_seg, a1,b1, a2,b2)
# -------------------------
@nb.njit(cache=True)
def _best_piecewise_1cp_bic(x, y, min_seg_points):
    n = x.size
    if n < 2 * min_seg_points:
        return 0, -1, 1e30, 1e30, 0.0, 0.0, 0.0, 0.0

    best_rss = 1e30
    best_cp = -1
    best_a1 = 0.0
    best_b1 = 0.0
    best_a2 = 0.0
    best_b2 = 0.0

    # enumerate cp positions
    for cp in range(min_seg_points, n - min_seg_points + 1):
        ok1, a1, b1, rss1, _, _, _ = _ols_fit(x[:cp], y[:cp])
        if ok1 == 0:
            continue
        ok2, a2, b2, rss2, _, _, _ = _ols_fit(x[cp:], y[cp:])
        if ok2 == 0:
            continue
        rss = rss1 + rss2
        if rss < best_rss:
            best_rss = rss
            best_cp = cp
            best_a1, best_b1 = a1, b1
            best_a2, best_b2 = a2, b2

    if best_cp < 0:
        return 0, -1, 1e30, 1e30, 0.0, 0.0, 0.0, 0.0

    # k=5: (a1,b1,a2,b2,cp) conservative complexity
    bic_seg = _bic_from_rss(best_rss, n, 5)
    return 1, best_cp, best_rss, bic_seg, best_a1, best_b1, best_a2, best_b2


# -------------------------
# Linear fit QC for S0:
# - intercept a > 0 (recommended)
# - slope b < 0 AND p_one < alpha
# - |b| >= bmin to avoid exploding S0
# returns (ok, a, b, s0, p_one, bic_lin)
# -------------------------
@nb.njit(cache=True)
def _linfit_s0_qc(x, y, alpha, bmin):
    ok, a, b, rss, se_b, t_b, p_one = _ols_fit(x, y)
    if ok == 0:
        return 0, 0.0, 0.0, np.nan, 1.0, 1e30

    n = x.size
    bic_lin = _bic_from_rss(rss, n, 2)

    # QC: intercept
    if (not np.isfinite(a)) or a <= 0.0:
        return 0, a, b, np.nan, p_one, bic_lin

    # QC: slope direction & magnitude
    if (not np.isfinite(b)) or b >= 0.0:
        return 0, a, b, np.nan, p_one, bic_lin
    if abs(b) < bmin:
        return 0, a, b, np.nan, p_one, bic_lin

    # QC: significance (one-sided)
    if (not np.isfinite(p_one)) or (p_one >= alpha):
        return 0, a, b, np.nan, p_one, bic_lin

    s0 = -a / b
    if (not np.isfinite(s0)) or (s0 <= 0.0):
        return 0, a, b, np.nan, p_one, bic_lin

    return 1, a, b, s0, p_one, bic_lin


@nb.njit(cache=True)
def _s0_one_pixel_qc(
    cwd, X, year_slices, nbins, q,
    min_event_steps, min_valid_bins, min_points_total, runaway_years,
    alpha, bmin, min_seg_points, delta_bic
):
    """
    One pixel S0 with QC outputs.
    Returns:
      s0(float32), p_one(float32), bic_lin(float32), bic_seg(float32), cp(int16), flag(uint8)

    flag:
      0 invalid/filtered
      1 linear accepted => S0 output
      2 segmented better (change point / flattening) => S0 not output
    """
    # runaway check
    if _count_runaway_years(cwd, year_slices) >= runaway_years:
        return np.float32(np.nan), np.float32(np.nan), np.float32(np.nan), np.float32(np.nan), np.int16(-1), np.uint8(0)

    T = cwd.size
    nY = year_slices.shape[0]

    # pooling arrays (max T points)
    pool_cwd = np.empty(T, dtype=np.float32)
    pool_x   = np.empty(T, dtype=np.float32)
    n_pool = 0
    gmax = 0.0

    # per-year largest event pooling
    for yi in range(nY):
        s = year_slices[yi, 0]
        e = year_slices[yi, 1]
        ok, st, ed, cwdmax = _event_window_one_year(cwd[s:e], min_event_steps)
        if ok == 0:
            continue

        # Add points from window
        for tt in range(s + st, s + ed):
            c = cwd[tt]
            x = X[tt]
            # key: only use c>0 to avoid reset/wet points
            if np.isfinite(c) and np.isfinite(x) and c > 0.0:
                pool_cwd[n_pool] = c
                pool_x[n_pool] = x
                n_pool += 1
                if c > gmax:
                    gmax = c

    if n_pool < min_points_total or (not np.isfinite(gmax)) or gmax <= 0.0:
        return np.float32(np.nan), np.float32(np.nan), np.float32(np.nan), np.float32(np.nan), np.int16(-1), np.uint8(0)

    # ===== binning: count -> regroup X by bin -> sort -> quantile
    counts = np.zeros(nbins, dtype=np.int32)
    bin_id = np.empty(n_pool, dtype=np.int32)

    for i in range(n_pool):
        b = int(pool_cwd[i] / gmax * nbins)
        if b < 0:
            b = 0
        elif b >= nbins:
            b = nbins - 1
        bin_id[i] = b
        counts[b] += 1

    offsets = np.empty(nbins, dtype=np.int32)
    pos = 0
    for b in range(nbins):
        offsets[b] = pos
        pos += counts[b]

    xgrp = np.empty(n_pool, dtype=np.float32)
    cursor = offsets.copy()
    for i in range(n_pool):
        b = bin_id[i]
        p = cursor[b]
        xgrp[p] = pool_x[i]
        cursor[b] += 1

    xb = np.empty(nbins, dtype=np.float32)
    yqv = np.empty(nbins, dtype=np.float32)
    n_valid = 0

    for b in range(nbins):
        n = counts[b]
        if n <= 0:
            continue
        st = offsets[b]
        ed = st + n

        xgrp[st:ed].sort()

        k = int(math.ceil(q * n) - 1)
        if k < 0:
            k = 0
        if k >= n:
            k = n - 1

        xq = xgrp[st + k]
        if not np.isfinite(xq):
            continue

        center = (b + 0.5) / nbins * gmax
        xb[n_valid] = center
        yqv[n_valid] = xq
        n_valid += 1

    if n_valid < min_valid_bins:
        return np.float32(np.nan), np.float32(np.nan), np.float32(np.nan), np.float32(np.nan), np.int16(-1), np.uint8(0)

    x_use = xb[:n_valid]
    y_use = yqv[:n_valid]

    # -------- (1) Linear fit QC --------
    ok, a, b, s0, p_one, bic_lin = _linfit_s0_qc(x_use, y_use, alpha, bmin)
    if ok == 0:
        # still return p_one & bic_lin if possible: do raw OLS once
        ok0, a0, b0, rss0, se_b0, t_b0, p0 = _ols_fit(x_use, y_use)
        if ok0 == 1:
            bic0 = _bic_from_rss(rss0, n_valid, 2)
            return np.float32(np.nan), np.float32(p0), np.float32(bic0), np.float32(np.nan), np.int16(-1), np.uint8(0)
        return np.float32(np.nan), np.float32(np.nan), np.float32(np.nan), np.float32(np.nan), np.int16(-1), np.uint8(0)

    # -------- (2) Piecewise 1 change-point BIC test --------
    ok2, cp, rss_seg, bic_seg, a1, b1, a2, b2 = _best_piecewise_1cp_bic(x_use, y_use, min_seg_points)
    if ok2 == 1:
        # if segmented clearly better: classify as change point / flattening => do not output S0
        # condition: bic_seg + delta_bic < bic_lin  (bic_seg substantially smaller)
        if bic_seg + delta_bic < bic_lin:
            return np.float32(np.nan), np.float32(p_one), np.float32(bic_lin), np.float32(bic_seg), np.int16(cp), np.uint8(2)
        else:
            return np.float32(s0), np.float32(p_one), np.float32(bic_lin), np.float32(bic_seg), np.int16(cp), np.uint8(1)

    # if no valid segmented fit
    return np.float32(s0), np.float32(p_one), np.float32(bic_lin), np.float32(np.nan), np.int16(-1), np.uint8(1)


@nb.njit(parallel=True, cache=True)
def s0_tile_qc(
    cwd3d, X3d, year_slices, nbins, q,
    min_event_steps, min_valid_bins, min_points_total, runaway_years,
    alpha, bmin, min_seg_points, delta_bic
):
    """
    cwd3d, X3d: (T, Y, X)

    Returns:
      s0_2d      float32 (Y, X)  (NaN if filtered)
      p_one_2d   float32 (Y, X)
      bic_lin_2d float32 (Y, X)
      bic_seg_2d float32 (Y, X)
      cp_2d      int16   (Y, X)
      flag_2d    uint8   (Y, X)
    """
    T, Y, X = cwd3d.shape

    s0_out = np.empty((Y, X), dtype=np.float32)
    p_out  = np.empty((Y, X), dtype=np.float32)
    bl_out = np.empty((Y, X), dtype=np.float32)
    bs_out = np.empty((Y, X), dtype=np.float32)
    cp_out = np.empty((Y, X), dtype=np.int16)
    fg_out = np.empty((Y, X), dtype=np.uint8)

    n_pix = Y * X
    for p in nb.prange(n_pix):
        j = p // X
        i = p - j * X
        s0, p_one, bic_lin, bic_seg, cp, flag = _s0_one_pixel_qc(
            cwd3d[:, j, i], X3d[:, j, i],
            year_slices, nbins, q,
            min_event_steps, min_valid_bins, min_points_total, runaway_years,
            alpha, bmin, min_seg_points, delta_bic
        )
        s0_out[j, i] = s0
        p_out[j, i]  = p_one
        bl_out[j, i] = bic_lin
        bs_out[j, i] = bic_seg
        cp_out[j, i] = cp
        fg_out[j, i] = flag

    return s0_out, p_out, bl_out, bs_out, cp_out, fg_out


# Backward-compatible: original behavior (S0 only)
@nb.njit(parallel=True, cache=True)
def s0_tile(
    cwd3d, X3d, year_slices, nbins, q,
    min_event_steps, min_valid_bins, min_points_total, runaway_years
):
    """
    Keeps original signature, returns only S0 (Y, X).
    Uses default QC hyperparameters suitable for EF/SIF_norm bin regression.
    """
    # Default QC params (tune as needed):
    alpha = 0.05
    bmin = 1e-8          # tune for your X scale
    min_seg_points = 5   # minimum bins per segment
    delta_bic = 2.0      # BIC improvement threshold

    s0_out, p_out, bl_out, bs_out, cp_out, fg_out = s0_tile_qc(
        cwd3d, X3d, year_slices, nbins, q,
        min_event_steps, min_valid_bins, min_points_total, runaway_years,
        alpha, bmin, min_seg_points, delta_bic
    )
    return s0_out


In [10]:
# 选一个 tile
y0,y1,x0,x1 = TILES[0]
cwd_t, def_t, dsif_t = load_tile(y0,y1,x0,x1)

# 运行 numba（第一次会编译，稍慢；后面很快）
s0_def  = s0_tile(cwd_t, def_t, YEAR_SLICES, NBINS, Q,
                  MIN_EVENT_STEPS, MINVALIDBINS, MINPOINTSTOTAL, RUNAWAY_YEARS)

s0_dsif = s0_tile(cwd_t, dsif_t, YEAR_SLICES, NBINS, Q,
                  MIN_EVENT_STEPS, MINVALIDBINS, MINPOINTSTOTAL, RUNAWAY_YEARS)

print("SdEF tile:", np.nanmin(s0_def), np.nanmax(s0_def), "nan%", np.isnan(s0_def).mean())
print("SdSIF tile:", np.nanmin(s0_dsif), np.nanmax(s0_dsif), "nan%", np.isnan(s0_dsif).mean())


SdEF tile: nan nan nan% 1.0
SdSIF tile: nan nan nan% 1.0


/tmp/ipykernel_3336617/670419628.py:12: RuntimeWarning: All-NaN slice encountered
  print("SdEF tile:", np.nanmin(s0_def), np.nanmax(s0_def), "nan%", np.isnan(s0_def).mean())
/tmp/ipykernel_3336617/670419628.py:13: RuntimeWarning: All-NaN slice encountered
  print("SdSIF tile:", np.nanmin(s0_dsif), np.nanmax(s0_dsif), "nan%", np.isnan(s0_dsif).mean())


In [ ]:
def write_tile(y0,y1,x0,x1, s0_def, s0_dsif):
    lat_tile = lat[y0:y1]
    lon_tile = lon[x0:x1]

    da_def = xr.DataArray(s0_def, dims=("lat","lon"),
                          coords={"lat": lat_tile, "lon": lon_tile},
                          name="SdEF").astype("float32")
    da_sif = xr.DataArray(s0_dsif, dims=("lat","lon"),
                          coords={"lat": lat_tile, "lon": lon_tile},
                          name="SdSIF").astype("float32")

    tile_tag = f"y{y0:04d}-{y1:04d}_x{x0:04d}-{x1:04d}"
    f_def = OUT_TILE_DIR / f"SdEF_{tile_tag}.nc"
    f_sif = OUT_TILE_DIR / f"SdSIF_{tile_tag}.nc"

    xr.Dataset({"SdEF": da_def}).to_netcdf(f_def, engine="netcdf4", format="NETCDF4",
                                          encoding={"SdEF": ENC_2D})
    xr.Dataset({"SdSIF": da_sif}).to_netcdf(f_sif, engine="netcdf4", format="NETCDF4",
                                            encoding={"SdSIF": ENC_2D})

    return f_def, f_sif

f1, f2 = write_tile(y0,y1,x0,x1, s0_def, s0_dsif)
print("Wrote:", f1, f2)


Wrote: /tera04/zhwei/xionghui/bedrock/data/out_tiles/SdEF_y0000-0300_x0000-0300.nc4 /tera04/zhwei/xionghui/bedrock/data/out_tiles/SdSIF_y0000-0300_x0000-0300.nc4


In [ ]:
def worker_one_tile(tile_id):
    y0,y1,x0,x1 = TILES[tile_id]
    tile_tag = f"y{y0:04d}-{y1:04d}_x{x0:04d}-{x1:04d}"
    f_def = OUT_TILE_DIR / f"SdEF_{tile_tag}.nc"
    f_sif = OUT_TILE_DIR / f"SdSIF_{tile_tag}.nc"

    # checkpoint: 已存在则跳过
    if f_def.exists() and f_sif.exists():
        return tile_id, "SKIP"

    # 每个进程独立打开（关键）
    ds_c = xr.open_dataset(CWD_PATH, engine="netcdf4")
    ds_e = xr.open_dataset(dEF_PATH, engine="netcdf4")
    ds_s = xr.open_dataset(dSIF_PATH, engine="netcdf4")

    cwd  = ds_c[VAR_CWD].isel(lat=slice(y0,y1), lon=slice(x0,x1)).transpose("time","lat","lon").load().data.astype(np.float32)
    dEF  = ds_e[VAR_dEF].isel(lat=slice(y0,y1), lon=slice(x0,x1)).transpose("time","lat","lon").load()
    dSIF = ds_s[VAR_dSIF].isel(lat=slice(y0,y1), lon=slice(x0,x1)).transpose("time","lat","lon").load()

    dEF  = dEF.where(dEF != FILL_X).data.astype(np.float32)
    dSIF = dSIF.where(dSIF != FILL_X).data.astype(np.float32)

    # compute
    s0_def  = s0_tile(cwd, dEF,  YEAR_SLICES, NBINS, Q,
                      MIN_EVENT_STEPS, MINVALIDBINS, MINPOINTSTOTAL, RUNAWAY_YEARS)
    s0_dsif = s0_tile(cwd, dSIF, YEAR_SLICES, NBINS, Q,
                      MIN_EVENT_STEPS, MINVALIDBINS, MINPOINTSTOTAL, RUNAWAY_YEARS)

    # write
    lat_tile = lat[y0:y1]
    lon_tile = lon[x0:x1]

    da_def = xr.DataArray(s0_def, dims=("lat","lon"),
                          coords={"lat": lat_tile, "lon": lon_tile},
                          name="SdEF").astype("float32")
    da_sif = xr.DataArray(s0_dsif, dims=("lat","lon"),
                          coords={"lat": lat_tile, "lon": lon_tile},
                          name="SdSIF").astype("float32")

    xr.Dataset({"SdEF": da_def}).to_netcdf(f_def, engine="netcdf4", format="NETCDF4",
                                           encoding={"SdEF": ENC_2D})
    xr.Dataset({"SdSIF": da_sif}).to_netcdf(f_sif, engine="netcdf4", format="NETCDF4",
                                            encoding={"SdSIF": ENC_2D})
    return tile_id, "OK"


# ====== 并行执行 ======
N_WORKERS = 24  # 你说至少24进程

done = 0
fail = 0

with ProcessPoolExecutor(max_workers=N_WORKERS) as ex:
    futs = {ex.submit(worker_one_tile, tid): tid for tid in range(len(TILES))}
    for fut in as_completed(futs):
        tid = futs[fut]
        try:
            tid2, status = fut.result()
            done += 1
            if status != "OK" and status != "SKIP":
                fail += 1
            if done % 10 == 0:
                print(f"[PROGRESS] {done}/{len(TILES)} done, fail={fail}")
        except Exception as e:
            fail += 1
            print(f"[FAIL] tile {tid}:", repr(e))

print("All done. fail=", fail)


[PROGRESS] 10/288 done, fail=0
[PROGRESS] 20/288 done, fail=0
[PROGRESS] 30/288 done, fail=0
[PROGRESS] 40/288 done, fail=0
[PROGRESS] 50/288 done, fail=0
[PROGRESS] 60/288 done, fail=0
[PROGRESS] 70/288 done, fail=0
[PROGRESS] 80/288 done, fail=0
[PROGRESS] 90/288 done, fail=0
[PROGRESS] 100/288 done, fail=0
[PROGRESS] 110/288 done, fail=0
[PROGRESS] 120/288 done, fail=0
[PROGRESS] 130/288 done, fail=0
[PROGRESS] 140/288 done, fail=0
[PROGRESS] 150/288 done, fail=0
[PROGRESS] 160/288 done, fail=0
[PROGRESS] 170/288 done, fail=0
[PROGRESS] 180/288 done, fail=0
[PROGRESS] 190/288 done, fail=0
[PROGRESS] 200/288 done, fail=0
[PROGRESS] 210/288 done, fail=0
[PROGRESS] 220/288 done, fail=0
[PROGRESS] 230/288 done, fail=0
[PROGRESS] 240/288 done, fail=0
[PROGRESS] 250/288 done, fail=0
[PROGRESS] 260/288 done, fail=0
[PROGRESS] 270/288 done, fail=0
[PROGRESS] 280/288 done, fail=0
All done. fail= 0


In [10]:
def assemble_global(varname, pattern_prefix, out_path):
    files = sorted(OUT_TILE_DIR.glob(f"{pattern_prefix}_y*-*_x*-*.nc"))
    if len(files) == 0:
        raise RuntimeError("No tile files found for " + pattern_prefix)

    # open_mfdataset 自动按坐标拼接（要求 tile 坐标不重叠且完整）
    ds = xr.open_mfdataset(files, combine="by_coords", engine="netcdf4")
    da = ds[varname].astype("float32")

    # 输出 chunk 可自行调
    enc = {varname: dict(zlib=True, complevel=4, dtype="float32", chunksizes=(600,600))}
    xr.Dataset({varname: da}).to_netcdf(out_path, engine="netcdf4", format="NETCDF4", encoding=enc)
    return out_path

out_def = assemble_global("SdEF", "SdEF", OUT_GLOBAL_DIR / "SdEF.nc")
out_sif = assemble_global("SdSIF", "SdSIF", OUT_GLOBAL_DIR / "SdSIF.nc")
print("Wrote global:", out_def, out_sif)


Wrote global: /tera04/zhwei/xionghui/bedrock/data/veg_activity/SdEF.nc /tera04/zhwei/xionghui/bedrock/data/veg_activity/SdSIF.nc


In [11]:
ds1 = xr.open_dataset(out_def)
ds2 = xr.open_dataset(out_sif)

SdEF  = ds1["SdEF"].values
SdSIF = ds2["SdSIF"].values

qc = {
    "SdEF": {
        "nan_frac": float(np.isnan(SdEF).mean()),
        "p01": float(np.nanpercentile(SdEF, 1)),
        "p50": float(np.nanpercentile(SdEF, 50)),
        "p99": float(np.nanpercentile(SdEF, 99)),
        "min": float(np.nanmin(SdEF)),
        "max": float(np.nanmax(SdEF)),
    },
    "SdSIF": {
        "nan_frac": float(np.isnan(SdSIF).mean()),
        "p01": float(np.nanpercentile(SdSIF, 1)),
        "p50": float(np.nanpercentile(SdSIF, 50)),
        "p99": float(np.nanpercentile(SdSIF, 99)),
        "min": float(np.nanmin(SdSIF)),
        "max": float(np.nanmax(SdSIF)),
    }
}
print(json.dumps(qc, indent=2))


{
  "SdEF": {
    "nan_frac": 0.8817354938271605,
    "p01": 25.294102001190186,
    "p50": 324.04734802246094,
    "p99": 24972.2095703125,
    "min": 0.004365758970379829,
    "max": 293556928.0
  },
  "SdSIF": {
    "nan_frac": 0.8932793981481482,
    "p01": 11.527358474731447,
    "p50": 348.12701416015625,
    "p99": 19929.036035156165,
    "min": 0.0007142720278352499,
    "max": 477670848.0
  }
}
